In [170]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [171]:
df_original = pd.read_csv('../bronze/raw_fighter_details.csv')
df_copy = df_original.copy()

In [172]:
df_original_stats = pd.read_csv('../bronze/ufc-fighters-statistics.csv')
df_stats_copy = df_original_stats.copy()

In [173]:
df_copy = df_copy.dropna()
df_copy

,fighter_name,Height,Weight,Reach,Stance,DOB,SLpM,Str_Acc,SApM,Str_Def,TD_Avg,TD_Acc,TD_Def,Sub_Avg
2,Shamil Abdurakhimov,"6' 3""",235 lbs.,"76""",Orthodox,"Sep 02, 1981",2.45,44%,2.45,58%,1.23,24%,47%,0.2
6,Daichi Abe,"5' 11""",170 lbs.,"71""",Orthodox,"Nov 27, 1991",3.80,33%,4.49,56%,0.33,50%,0%,0.0
8,Klidson Abreu,"6' 0""",205 lbs.,"74""",Orthodox,"Dec 24, 1992",2.05,40%,2.90,55%,0.64,20%,80%,0.0
11,Juan Adams,"6' 5""",265 lbs.,"80""",Orthodox,"Jan 16, 1992",7.09,55%,4.06,34%,0.91,66%,57%,0.0
12,Anthony Adams,"6' 1""",185 lbs.,"76""",Orthodox,"Jan 13, 1988",3.17,41%,5.93,44%,0.00,0%,0%,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3581,Zhalgas Zhumagulov,"5' 4""",125 lbs.,"66""",Switch,"Aug 29, 1988",4.17,49%,4.00,58%,1.50,21%,33%,0.0
3582,Fares Ziam,"6' 1""",155 lbs.,"75""",Orthodox,"Mar 21, 1997",1.90,47%,1.37,66%,0.50,11%,65%,0.0
3586,Cat Zingano,"5' 6""",145 lbs.,"68""",Southpaw,"Jul 01, 1982",2.57,61%,1.63,47%,2.77,65%,42%,0.8
3591,Zhang Tiequan,"5' 8""",155 lbs.,"69""",Orthodox,"Jul 25, 1978",1.23,36%,2.14,51%,1.95,58%,75%,3.4


In [174]:
def convert_height(height):
    feet, inches = map(int, height.replace("\"", "").split("'"))
    total_inches = feet * 12 + inches
    total_cm = total_inches * 2.54
    return total_cm

df_copy['Height']=df_copy['Height'].apply(convert_height)
df_copy= df_copy.rename(columns={'Height': 'Height_cm'})


In [175]:
df_copy['Weight'] = df_copy['Weight'].str.replace('lbs.', '')
df_copy['Weight'] = df_copy['Weight'].astype(float)
df_copy['Weight'] = df_copy['Weight'] * 0.45
df_copy= df_copy.rename(columns={'Weight': 'Weight_kg'})


In [176]:
df_copy['Reach'] = pd.to_numeric(df_copy['Reach'].str.replace('"', '')) * 2.54
df_copy= df_copy.rename(columns={'Reach': 'Reach_cm'})


In [177]:
ohe_columns = ['Stance']
ohe_stance = OneHotEncoder(sparse_output=False)
ohe_stance.fit(df_copy[ohe_columns])
ohe_column_name = ohe_stance.get_feature_names_out(ohe_columns)
df_copy[ohe_column_name] = ohe_stance.transform(df_copy[ohe_columns])

In [178]:
df_copy['DOB'] = pd.to_datetime(df_copy['DOB'])
df_copy['DOB'] = df_copy['DOB'].dt.strftime('%d-%m-%Y')

In [181]:
df_copy

,fighter_name,Height_cm,Weight_kg,Reach_cm,Stance,DOB,SLpM,Str_Acc,SApM,Str_Def,TD_Avg,TD_Acc,TD_Def,Sub_Avg,Stance_Open Stance,Stance_Orthodox,Stance_Southpaw,Stance_Switch
2,Shamil Abdurakhimov,190.50,105.75,193.04,Orthodox,02-09-1981,2.45,44%,2.45,58%,1.23,24%,47%,0.2,0.0,1.0,0.0,0.0
6,Daichi Abe,180.34,76.50,180.34,Orthodox,27-11-1991,3.80,33%,4.49,56%,0.33,50%,0%,0.0,0.0,1.0,0.0,0.0
8,Klidson Abreu,182.88,92.25,187.96,Orthodox,24-12-1992,2.05,40%,2.90,55%,0.64,20%,80%,0.0,0.0,1.0,0.0,0.0
11,Juan Adams,195.58,119.25,203.20,Orthodox,16-01-1992,7.09,55%,4.06,34%,0.91,66%,57%,0.0,0.0,1.0,0.0,0.0
12,Anthony Adams,185.42,83.25,193.04,Orthodox,13-01-1988,3.17,41%,5.93,44%,0.00,0%,0%,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3581,Zhalgas Zhumagulov,162.56,56.25,167.64,Switch,29-08-1988,4.17,49%,4.00,58%,1.50,21%,33%,0.0,0.0,0.0,0.0,1.0
3582,Fares Ziam,185.42,69.75,190.50,Orthodox,21-03-1997,1.90,47%,1.37,66%,0.50,11%,65%,0.0,0.0,1.0,0.0,0.0
3586,Cat Zingano,167.64,65.25,172.72,Southpaw,01-07-1982,2.57,61%,1.63,47%,2.77,65%,42%,0.8,0.0,0.0,1.0,0.0
3591,Zhang Tiequan,172.72,69.75,175.26,Orthodox,25-07-1978,1.23,36%,2.14,51%,1.95,58%,75%,3.4,0.0,1.0,0.0,0.0


In [180]:
df_copy['Stance'].unique()

array(['Orthodox', 'Switch', 'Southpaw', 'Open Stance'], dtype=object)